# Fully Connected Neural Network

Our first trial would be to use Keras to develop a fully connected neural network with a single hidden layer. So we will start with the most basic layout of a neural network and see how it works for this problem

In [1]:
# Common python import statements
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import  Dense, Flatten
import numpy as np
import math


# fixed random seed to have consistent results
np.random.seed(123)

# dimensions of our images. Keras will be resizing all images to have consistent data fed to training and testing phases
img_width, img_height = 300, 300 

train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples = 3000
nb_validation_samples = 300
epochs = 5
batch_size = 400 # in next steps we cannot just use any size here as it will be limited by GPU memory size.
# if you get OOM exception anytime, decrease this value

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# We need do define input data format or shape. Tensorflow defaults for image are height then width then colour channels
# height and width are the same here but can have some implications when they are different and when doing prediction (inference)

input_shape = (img_height, img_width, 3)

## Define Keras model

In [3]:
model = Sequential() # sequential mode is a stack of layers one after other and this is the simplest network layout
model.add(Dense(3, input_shape=input_shape)) # this is input layer apparently
model.add(Flatten()) # flatten the multi dimensional input into a single dimension (vector)
model.add(Dense(32, activation = 'relu')) # RELU is simply max(x,0) which adds some non linear activation
model.add(Dense(1, activation = 'sigmoid')) # softmax is used in output layer only

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 300, 300, 3)       12        
_________________________________________________________________
flatten_1 (Flatten)          (None, 270000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                8640032   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 8,640,077
Trainable params: 8,640,077
Non-trainable params: 0
_________________________________________________________________


**Can you see above that a simple network for 300x300 image has more than 8 million weights to train. Totally crazy !!**

For any model we need to compile it first with a loss function based on the problem and an optimizer and a performance metric.
I can just throw two words on accuracy here. If we are doing dogs vs cats then we may be concerned with 
total number of correct predictions only. But what if we are doing cancer detection where the percentage of 
positives is something like 1%. Clearly we do not want to inform someone you have cancer when he does not and also
we do not want to miss people with cancer and let them go home without informing them. So for such cases there are other 
metrics like recall and precision. Just google "F1 Score" for more insights.

In [5]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Next we need to prepare some generators that can feed Keras with data for training and testing. They shield us from the hard task of loading images and converting them to multi dimensional arrays and so on. Also sometimes training datasets could be small so those generators can do some process called data augmentation to fake new training data. For example, images could be rotated a bit or flipped as long as they contain the same main patterns needed for training. 

*Side note:*
Scaling to 1/255 below is very common in many vision problems and appears in most of preprocessing steps of major commonly used models like VGG, Inception & RESNET.

In [6]:
# this is the augmentation configuration we will use for training, some scaling and image manipulation as well
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing, only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3000 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


Next step we will train the network and adjusting the weights. Output of final training epoch will be final accuracy for the few epochs run. An epoch is a training run against all training data that is spearated into a number of steps in which each step will be training a specific number of training samples of size (batch size).

In [7]:
model.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(nb_train_samples / batch_size),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=math.ceil(nb_validation_samples/batch_size))

Epoch 1/5
8/8 [==============================] - 65s 8s/step - loss: 7.0299 - acc: 0.4930 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 2/5
8/8 [==============================] - 63s 8s/step - loss: 8.1128 - acc: 0.4967 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 3/5
8/8 [==============================] - 60s 8s/step - loss: 8.0859 - acc: 0.4983 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/5
8/8 [==============================] - 60s 8s/step - loss: 8.0501 - acc: 0.5006 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/5
8/8 [==============================] - 60s 8s/step - loss: 8.0367 - acc: 0.5014 - val_loss: 8.0590 - val_acc: 0.5000


The above is a very bad result, you know why? A binary classification problem with a dummy solver will give 50% accuracy if the solver picks one of the two classes all the time assuming the dataset is balanced.
Let's see in notebook #3 if we can do better by adding another hidden layer or maybe more.